<a href="https://colab.research.google.com/github/Devica2000/StanfordTech16LLM/blob/main/Summarize%20Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.9 MB/s eta 0:00:00


## Homeowork Task 1
Summarize some text

In [9]:
from openai import OpenAI
from google.colab import userdata

open_ai_key = userdata.get('open_ai_key')

client = OpenAI(api_key=open_ai_key)

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "Summarize the text into a headline"},
    {"role": "user", "content": """
          Meta will reward shareholders with its first-ever dividend and an additional $50bn in share buybacks
        as bumper fourth-quarter results sent its shares up by more than 14 per cent in after-hours trading
        on Thursday.
        """}
  ]
)

summary = response.choices[0].message.content
print(summary)

Meta announces first dividend and $50bn in share buybacks after strong fourth-quarter results


## What am I doing?

My goal is to use the chat completion API to summarize a song I provide as input. to achieve this, I need to convert the song file (.m4a) into text to form a transcript of its lyrics which can then be given as input to the chat completion.

I collected data in 2 ways:
1. I played the song and recorded the audio on my phone. (Don't Give Up On Me by Andy Grammer)
2. I recited the lyrics of the song 'Stargazing' by Myles Smith on my own and recorded my audio (without any background music, instruments etc.)

To convert audio to text, I tried the following:
1. Use the SpeechRecongnition in python
2. Use pydub and Google cloud speech
3. Use Whisper by OpenAI

The first two methods worked fine with the audio that had my voice and no music. However, Whisper was completely able to convert the song (with background noise, music etc) to text (with some degree of error).

In [11]:
%pip install pydub google-cloud-speech google-cloud-storage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.4/289.4 kB 15.3 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File

In [28]:
from pydub import AudioSegment
from google.cloud import speech
from google.cloud import storage
import os
import wave

# Set up Google Cloud authentication
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "service_acc_file.json"

# Convert .m4a to .wav
def convert_m4a_to_wav(m4a_path, wav_path):
    audio = AudioSegment.from_file(m4a_path, format="m4a")
    audio.export(wav_path, format="wav")

# Get sample rate from .wav file
def get_sample_rate(wav_path):
    with wave.open(wav_path, "rb") as wav_file:
        return wav_file.getframerate()

# Upload file to Google Cloud Storage
def upload_to_gcs(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    return f'gs://{bucket_name}/{destination_blob_name}'

# Transcribe the audio from GCS URI using LongRunningRecognize
def transcribe_audio_gcs_longrunning(gcs_uri, sample_rate_hertz):
    client = speech.SpeechClient()

    audio = speech.RecognitionAudio(uri=gcs_uri)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=sample_rate_hertz,  # Use the actual sample rate of the audio file
        language_code="en-US"
    )

    operation = client.long_running_recognize(config=config, audio=audio)
    response = operation.result(timeout=900)

    transcript = ""
    for result in response.results:
        transcript += result.alternatives[0].transcript

    return transcript

# Paths and bucket details
m4a_path = "song_speech.m4a"
wav_path = "song_audio.wav"
bucket_name = "tech16llmsong"
destination_blob_name = "song_audio_gcp.wav"

# Convert and upload
convert_m4a_to_wav(m4a_path, wav_path)
sample_rate_hertz = get_sample_rate(wav_path)
gcs_uri = upload_to_gcs(bucket_name, wav_path, destination_blob_name)
print(f"File uploaded to {gcs_uri}")

# Transcribe
transcript = transcribe_audio_gcs_longrunning(gcs_uri, sample_rate_hertz)
print("Transcript:", transcript)


File uploaded to gs://tech16llmsong/song_audio_gcp.wav
Transcript: time Stood Still just like a photograph you made me feel like this would last forever looking in your eyes I see my whole life they say you know it when you know it and I know promise that you'll hold me close don't let me go hey take my heart don't break it love me to the bones all this time I've wasted you were right there or you and I stargazing intertwining Souls we will never ever strangers I lose my mind when I'm around you how come I'm alive but feel so lost I want to give you all I've got this and I know that you'll hold me close don't let me go take my heart don't break it love me to my bones all this time I've wasted you were right there all along you and I stargazing intertwining Souls even never strangers you were right there all along all along


In [29]:
from openai import OpenAI
from google.colab import userdata

open_ai_key = userdata.get('open_ai_key')

client = OpenAI(api_key=open_ai_key)

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "Summarize the text. Understand the text thoroughly, and then give your output."},
    {"role": "user", "content": transcript}
  ]
)

summary = response.choices[0].message.content
print("Summary of the input text:\n", summary)

Summary of the input text:
 The text describes a deep connection and love between two people, where time seems to stand still and they feel like they are meant to be together forever. The speaker expresses their desire for the other person to hold them close, not let them go, and love them completely. The lyrics convey a sense of longing, appreciation, and a strong emotional bond that transcends time.


In [8]:
%pip install SpeechRecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 38.6 MB/s eta 0:00:00


In [30]:
import speech_recognition as sr
# Initialize recognizer class
r = sr.Recognizer()
# audio object
audio = sr.AudioFile("song_audio.wav")
#read audio object and transcribe
with audio as source:
    audio = r.record(source)
    result = r.recognize_google(audio)

print(result)

time Stood Still just like a photograph you made me feel like this would last forever looking in your eyes I see my whole life they say you know it when you know it and I know oh oh oh oh promise that you'll hold me close don't let me go hey take my heart don't break it love me to the bones all this time I wasted you were right there all along you and I start kissing intertwining Souls we will never ever strangers you were right there all alone I lose my mind when I'm around you how come I am alive when I'm without you I can't help but feel so lost I want to give you all I've got they say you know it when you know it and I know oh oh oh oh promise that you'll hold me close don't let me go take my heart don't break it love me to my bones all this time I wasted you were right there all along you and I start kissing intertwining Souls we were never strangers you were right there all along all along


In [31]:
from openai import OpenAI
from google.colab import userdata

open_ai_key = userdata.get('open_ai_key')

client = OpenAI(api_key=open_ai_key)

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "Summarize the text. Understand the text thoroughly, and then give your output."},
    {"role": "user", "content": result}
  ]
)

summary = response.choices[0].message.content
print("Summary of the input text:\n", summary)

Summary of the input text:
 The text is about a strong emotional connection and love between two people. The narrator expresses feelings of time standing still when with their partner, feeling alive and complete, and being deeply in love. There is a sense of longing and commitment, with a promise to hold on and not let go. The lyrics emphasize the idea that the two souls were always meant to be together, even if they didn't realize it at first. The message conveys the idea of finding true love and wanting to give everything to the other person.


In [2]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-o012c83a
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-o012c83a
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-

In [5]:
import whisper

model = whisper.load_model("small")
result = model.transcribe("DontGiveUpOnMe.m4a")
song_transcript = result["text"]
print(song_transcript)
print(result["text"])

100%|███████████████████████████████████████| 461M/461M [00:07<00:00, 65.2MiB/s]


 I will fight, I will fight for you. I always do until my eyes black and blue. I will stay, I will stay with you. We'll make it to the other side like lovers do. I'll reach my hands out in the dark and wait for yours to enter like a wave for you. A wave for you. I'll give it up, I'll not give it up, give it up, no no yeah. Even when I'm down on my last plan, even when they say there's nothing left. So don't give up, I'll not give it up, I'm not giving up, give it up, no no no yeah. Even when nobody else believes, I'm not going down that easily. So don't give up on me. Now I will hold, I'll hold on to you. No matter where I go, I won't shake me loose. I'll reach my hands out in the dark and wait for yours to enter like a wave for you. A wave for you. I'll reach my hands out in the dark and wait for yours to enter like a wave for you. I'll reach my hands out in the dark and wait for yours to enter like a wave for you. I'll not give up, I'll not give it up, give it up, no no no yeah. Even

In [10]:
from openai import OpenAI
from google.colab import userdata

open_ai_key = userdata.get('open_ai_key')

client = OpenAI(api_key=open_ai_key)

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "Summarize the text. Understand the text thoroughly, and then give your output."},
    {"role": "user", "content": song_transcript}
  ]
)

summary = response.choices[0].message.content
print("Summary of the song Don't Give Up On Me:\n", summary)

Summary of the song Don't Give Up On Me:
 The text is about the speaker's determination to support and stand by someone they care about. They vow to fight, never give up, and always be there, even in difficult times. The speaker emphasizes not giving up and holding on to the relationship, no matter what challenges come their way. They express a strong commitment to the other person and urge them not to give up.


In [4]:
#whisper "DontGiveUpOnMe.m4a" --model medium


100%|█████████████████████████████████████| 1.42G/1.42G [00:39<00:00, 38.3MiB/s]
Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: English
[00:00.000 --> 00:19.200]  I will fight, I will fight for you I always do until my heart is black and blue
[00:19.200 --> 00:36.400]  I will stay, I will stay with you We'll make it to the other side, like lovers do
[00:36.400 --> 00:53.600]  I'll reach my hands out in the dark and wait for yours to interlock I'll wait for you, I'll wait for you
[00:53.600 --> 01:10.800]  I'm not giving up, I'm not giving up, giving up no not yet Even when I'm down on my last breath Even when they say there's nothing left So don't give up on me
[01:10.800 --> 01:28.800]  I'm not giving up, I'm not giving up, giving up no not yet Even when nobody else believes I'm not going down that easily So don't give up on me
[01:29.800 --> 01:48.000]  Now I won't hold, I'll hold on to you No matter what this world wi